In [2]:
# import of libraries we will use
%matplotlib inline
import random
#import tsfresh
import os
import math
from scipy import stats
from scipy.spatial.distance import pdist
from math import sqrt, log, floor
from fastdtw import fastdtw
import ipywidgets as widgets
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
import pandas as pd
import seaborn as sns
#Librerias clustering
from statistics import mean
from scipy.spatial.distance import euclidean
import scipy.cluster.hierarchy as hac
from scipy.cluster.hierarchy import fcluster
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN
from sklearn.manifold import TSNE
from sklearn.metrics import normalized_mutual_info_score, adjusted_rand_score, silhouette_score, silhouette_samples
from sklearn.metrics import mean_squared_error
from scipy.spatial import distance
#Librerías LSTM
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from keras.utils.vis_utils import plot_model

#Ajustes gráficos
sns.set(style='white')
# "Fijar" randomnes para reproducibilidad
random.seed(42)

Leemos los datos 

In [3]:
path = "./data/Notificaciones/"
filename_read = os.path.join(path,"normalizado.csv")
dataset = pd.read_csv(filename_read,delimiter=",",engine='python')
dataset

,Unnamed: 0,anio,casos,distrito_nombre,distrito_notif,incidencia,semana,0
0,0,2009.0,0.0,LOMA PLATA,0,0.0,1.0,NaN
1,1,2009.0,0.0,LOMA PLATA,0,0.0,2.0,NaN
2,2,2009.0,0.0,LOMA PLATA,0,0.0,3.0,NaN
3,3,2009.0,0.0,LOMA PLATA,0,0.0,4.0,NaN
4,4,2009.0,0.0,LOMA PLATA,0,0.0,5.0,NaN
...,...,...,...,...,...,...,...,...
57500,48,2013.0,0.0,JUAN EULOGIO ESTIGARRIBIA,0,0.0,49.0,NaN
57501,49,2013.0,0.0,JUAN EULOGIO ESTIGARRIBIA,0,0.0,50.0,NaN
57502,50,2013.0,0.0,JUAN EULOGIO ESTIGARRIBIA,0,0.0,51.0,NaN
57503,51,2013.0,0.0,JUAN EULOGIO ESTIGARRIBIA,0,0.0,52.0,NaN


In [4]:
print("Nulos ",dataset['incidencia'].isna().sum())

Nulos  21306


In [5]:
dataset = dataset.replace([np.inf, -np.inf], np.nan).fillna(0.000001)

In [6]:
print("Nulos ",dataset['incidencia'].isna().sum())

Nulos  0


## Por distrito

In [7]:
listaDist = dataset['distrito_nombre'].tolist()
listaDist.sort()
listaDist = list(dict.fromkeys(listaDist))
print('Son ', len(listaDist), ' distritos')
print(listaDist)

Son  217  distritos
['1RO DE MARZO', '25 DE DICIEMBRE', '3 DE FEBRERO', 'ABAI', 'ACAHAY', 'ALBERDI', 'ALTO VERA', 'ALTOS', 'ANTEQUERA', 'AREGUA', 'ARROYOS Y ESTEROS', 'ASUNCION', 'ATYRA', 'AYOLAS', 'AZOTEY', 'BAHIA NEGRA', 'BELEN', 'BELLA VISTA', 'BENJAMIN ACEVAL', 'BORJA', 'BUENA VISTA', 'CAACUPE', 'CAAGUAZU', 'CAAZAPA', 'CABALLERO ALVAREZ', 'CAMBYRETA', 'CAPIATA', 'CAPIIBARY', 'CAPITAN BADO', 'CAPITAN MEZA', 'CAPITAN MIRANDA', 'CARAGUATAY', 'CARAPEGUA', 'CARAYAO', 'CARLOS ANTONIO LOPEZ', 'CARMELO PERALTA', 'CARMEN DEL PARANA', 'CECILIO BAEZ', 'CERRITO', 'CHACO', 'CHORE', 'COLONIA FRAM', 'COLONIA INDEPENDENCIA', 'CONCEPCION', 'CORONEL BOGADO', 'CORONEL MARTINEZ', 'CORONEL OVIEDO', 'CORPUS CHRISTI', 'CURUGUATY', 'DESMOCHADOS', 'DR BOTRELL', 'DR. JUAN MANUEL FRUTOS', 'EDELIRA', 'EMBOSCADA', 'ENCARNACION', 'ESCOBAR', 'EUGENIO A GARAY', 'EUSEBIO AYALA', 'FASSARDI', 'FELIX PEREZ CARDOZO', 'FERNANDO DE LA MORA', 'FILADELFIA', 'FUERTE OLIMPO', 'GENERAL AQUINO', 'GENERAL ARTIGAS', 'GENERAL BE

In [8]:
#CIUDAD DEL ESTE

In [9]:
listaDist=['ASUNCION','FERNANDO DE LA MORA','SAN LORENZO','FUERTE OLIMPO','PEDRO JUAN CABALLERO','FILADELFIA','CORONEL OVIEDO','CAAZAPA','SALTO DEL GUAIRA',
          'AREGUA','CONCEPCION','CAACUPE','VILLARRICA','ENCARNACION','SAN JUAN BAUTISTA','PILAR','PARAGUARI','VILLA HAYES','SAN PEDRO DEL YCUAMANDIYU']

Función para crear el dataset según LSTM. Convierte un array de valores en una matriz

In [10]:
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
    return np.array(dataX), np.array(dataY)

In [11]:
def distLSTM(dataset):
    #Establecer un seed para reproducir después
    np.random.seed(45)
    # Arreglar el dataset
    dataframe=dataset[['incidencia']]
    dataset = dataframe.values
    dataset = dataset.astype('float32')
    # normalize the dataset
    scaler = MinMaxScaler(feature_range=(0, 1))
    dataset = scaler.fit_transform(dataset)
    # split into train and test sets
    train_size = int(len(dataset) * 0.67)
    test_size = len(dataset) - train_size
    train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]
    print(len(train), len(test))
    # reshape into X=t and Y=t+1
    look_back = 1
    trainX, trainY = create_dataset(train, look_back)
    testX, testY = create_dataset(test, look_back)
    # reshape input to be [samples, time steps, features]
    trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
    testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))
    # create and fit the LSTM network
    model = Sequential()
    model.add(LSTM(4, input_shape=(1, look_back)))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    history = model.fit(trainX, trainY,validation_data=(testX, testY), epochs=500, batch_size=1, verbose=2)
    train_mse = model.evaluate(trainX, trainY, verbose=0)
    test_mse = model.evaluate(testX, testY, verbose=0)
    print('Train: %.3f, Test: %.3f' % (train_mse, test_mse))
    # plot loss during training
    #plt.figure(figsize=(6,3))
    #plt.title('Loss / Mean Squared Error')
    #plt.plot(history.history['loss'], label='train')
    #plt.plot(history.history['val_loss'], label='test')
    #plt.legend()
    #plt.legend()
    #plt.savefig('./data/Resultados/Distritos/loss_'+str(dist)+'.png')
    #plt.show()
    # make predictions
    trainPredict = model.predict(trainX)
    testPredict = model.predict(testX)
    # invert predictions
    trainPredict = scaler.inverse_transform(trainPredict)
    trainY = scaler.inverse_transform([trainY])
    testPredict = scaler.inverse_transform(testPredict)
    testY = scaler.inverse_transform([testY])
    # calculate root mean squared error
    trainScore = math.sqrt(mean_squared_error(trainY[0], trainPredict[:,0]))
    #print('Train Score: %.2f RMSE' % (trainScore))
    testScore = math.sqrt(mean_squared_error(testY[0], testPredict[:,0]))
    #print('Test Score: %.2f RMSE' % (testScore))
    #Guardar modelo
    model.save(str('./data/Resultados/Clusters/model_'+str(dist)+'.h5'))
    print('Modelo de',dist,'guardado')
    # shift train predictions for plotting
    trainPredictPlot = np.empty_like(dataset)
    trainPredictPlot[:, :] = np.nan
    trainPredictPlot[look_back:len(trainPredict)+look_back, :] = trainPredict
    # shift test predictions for plotting
    testPredictPlot = np.empty_like(dataset)
    testPredictPlot[:, :] = np.nan
    testPredictPlot[len(trainPredict)+(look_back*2)+1:len(dataset)-1, :] = testPredict
        
    #Para generar el dataframe
    reg=len(dataframe.index)
    # plot baseline and predictions
    #plt.figure(figsize=(10,3))
    #plt.xlabel('Weeks')
    #plt.ylabel('Number of confirmed cases')
    #plt.plot(scaler.inverse_transform(dataset), label="Real")

    #train=plt.plot(trainPredictPlot, label="Train")
    #predict=plt.plot(testPredictPlot, label="Predict")

    #plt.legend()
    #plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)


    #plt.show()
    #plt.savefig('./data/Resultados/Distritos/predict_'+str(dist)+'.png')
    return trainScore,testScore,reg

In [12]:
MSEtrain=[]
MSETest=[]
Registros=[]
Dist=[]
for dist in listaDist:
    #print(dist)
    distrito=dataset['distrito_nombre']==dist
    datasetxdist=dataset[distrito]
    print("Distrito: ",dist)
    trainScore,testScore,reg=distLSTM(datasetxdist)
    print("trainScore",trainScore)
    print("testScore",testScore)
    print("reg",reg)
    #Para el dataframe
    MSEtrain.append(trainScore)
    MSETest.append(testScore)
    Registros.append(reg)
    Dist.append(dist)
    data_tuples = list(zip(Dist,MSEtrain,MSETest,Registros))
    Tabla = pd.DataFrame(data_tuples, columns =['Distrito', 'Train Score', 'Test Score', 'Registros']) 

Distrito:  ASUNCION
177 88
Epoch 1/500
175/175 - 3s - loss: 0.0051 - val_loss: 0.0667 - 3s/epoch - 17ms/step
Epoch 2/500
175/175 - 0s - loss: 0.0049 - val_loss: 0.0639 - 469ms/epoch - 3ms/step
Epoch 3/500
175/175 - 1s - loss: 0.0047 - val_loss: 0.0617 - 528ms/epoch - 3ms/step
Epoch 4/500
175/175 - 1s - loss: 0.0046 - val_loss: 0.0618 - 590ms/epoch - 3ms/step
Epoch 5/500
175/175 - 1s - loss: 0.0045 - val_loss: 0.0586 - 651ms/epoch - 4ms/step
Epoch 6/500
175/175 - 1s - loss: 0.0044 - val_loss: 0.0559 - 539ms/epoch - 3ms/step
Epoch 7/500
175/175 - 1s - loss: 0.0041 - val_loss: 0.0539 - 536ms/epoch - 3ms/step
Epoch 8/500
175/175 - 1s - loss: 0.0038 - val_loss: 0.0475 - 513ms/epoch - 3ms/step
Epoch 9/500
175/175 - 0s - loss: 0.0035 - val_loss: 0.0433 - 433ms/epoch - 2ms/step
Epoch 10/500
175/175 - 1s - loss: 0.0031 - val_loss: 0.0368 - 682ms/epoch - 4ms/step
Epoch 11/500
175/175 - 1s - loss: 0.0026 - val_loss: 0.0316 - 692ms/epoch - 4ms/step
Epoch 12/500
175/175 - 1s - loss: 0.0022 - val_lo

Epoch 94/500
175/175 - 0s - loss: 4.8844e-04 - val_loss: 0.0070 - 376ms/epoch - 2ms/step
Epoch 95/500
175/175 - 0s - loss: 4.7359e-04 - val_loss: 0.0070 - 402ms/epoch - 2ms/step
Epoch 96/500
175/175 - 0s - loss: 4.8579e-04 - val_loss: 0.0072 - 368ms/epoch - 2ms/step
Epoch 97/500
175/175 - 0s - loss: 4.9275e-04 - val_loss: 0.0070 - 369ms/epoch - 2ms/step
Epoch 98/500
175/175 - 0s - loss: 4.8355e-04 - val_loss: 0.0070 - 356ms/epoch - 2ms/step
Epoch 99/500
175/175 - 0s - loss: 4.7649e-04 - val_loss: 0.0070 - 396ms/epoch - 2ms/step
Epoch 100/500
175/175 - 0s - loss: 4.8164e-04 - val_loss: 0.0071 - 399ms/epoch - 2ms/step
Epoch 101/500
175/175 - 0s - loss: 4.8446e-04 - val_loss: 0.0070 - 447ms/epoch - 3ms/step
Epoch 102/500
175/175 - 0s - loss: 4.9003e-04 - val_loss: 0.0071 - 388ms/epoch - 2ms/step
Epoch 103/500
175/175 - 0s - loss: 4.9839e-04 - val_loss: 0.0070 - 418ms/epoch - 2ms/step
Epoch 104/500
175/175 - 0s - loss: 4.8436e-04 - val_loss: 0.0070 - 372ms/epoch - 2ms/step
Epoch 105/500
17

175/175 - 1s - loss: 4.6157e-04 - val_loss: 0.0076 - 714ms/epoch - 4ms/step
Epoch 186/500
175/175 - 1s - loss: 4.5552e-04 - val_loss: 0.0077 - 582ms/epoch - 3ms/step
Epoch 187/500
175/175 - 1s - loss: 4.5463e-04 - val_loss: 0.0078 - 628ms/epoch - 4ms/step
Epoch 188/500
175/175 - 1s - loss: 4.7956e-04 - val_loss: 0.0078 - 509ms/epoch - 3ms/step
Epoch 189/500
175/175 - 0s - loss: 4.5907e-04 - val_loss: 0.0078 - 436ms/epoch - 2ms/step
Epoch 190/500
175/175 - 0s - loss: 4.5110e-04 - val_loss: 0.0076 - 468ms/epoch - 3ms/step
Epoch 191/500
175/175 - 0s - loss: 4.6684e-04 - val_loss: 0.0079 - 469ms/epoch - 3ms/step
Epoch 192/500
175/175 - 0s - loss: 4.6146e-04 - val_loss: 0.0079 - 464ms/epoch - 3ms/step
Epoch 193/500
175/175 - 1s - loss: 4.6423e-04 - val_loss: 0.0076 - 535ms/epoch - 3ms/step
Epoch 194/500
175/175 - 1s - loss: 4.8155e-04 - val_loss: 0.0077 - 816ms/epoch - 5ms/step
Epoch 195/500
175/175 - 1s - loss: 4.6171e-04 - val_loss: 0.0077 - 561ms/epoch - 3ms/step
Epoch 196/500
175/175 - 

Epoch 277/500
175/175 - 0s - loss: 4.7949e-04 - val_loss: 0.0085 - 460ms/epoch - 3ms/step
Epoch 278/500
175/175 - 1s - loss: 4.6430e-04 - val_loss: 0.0087 - 793ms/epoch - 5ms/step
Epoch 279/500
175/175 - 1s - loss: 4.6037e-04 - val_loss: 0.0085 - 934ms/epoch - 5ms/step
Epoch 280/500
175/175 - 1s - loss: 4.5424e-04 - val_loss: 0.0087 - 823ms/epoch - 5ms/step
Epoch 281/500
175/175 - 1s - loss: 4.4997e-04 - val_loss: 0.0086 - 657ms/epoch - 4ms/step
Epoch 282/500
175/175 - 1s - loss: 4.5239e-04 - val_loss: 0.0084 - 615ms/epoch - 4ms/step
Epoch 283/500
175/175 - 1s - loss: 4.4326e-04 - val_loss: 0.0085 - 550ms/epoch - 3ms/step
Epoch 284/500
175/175 - 0s - loss: 4.4842e-04 - val_loss: 0.0085 - 481ms/epoch - 3ms/step
Epoch 285/500
175/175 - 1s - loss: 4.6881e-04 - val_loss: 0.0084 - 593ms/epoch - 3ms/step
Epoch 286/500
175/175 - 0s - loss: 4.5388e-04 - val_loss: 0.0084 - 465ms/epoch - 3ms/step
Epoch 287/500
175/175 - 0s - loss: 4.4933e-04 - val_loss: 0.0086 - 490ms/epoch - 3ms/step
Epoch 288/

175/175 - 1s - loss: 4.5385e-04 - val_loss: 0.0092 - 632ms/epoch - 4ms/step
Epoch 369/500
175/175 - 1s - loss: 4.4495e-04 - val_loss: 0.0091 - 575ms/epoch - 3ms/step
Epoch 370/500
175/175 - 1s - loss: 4.4531e-04 - val_loss: 0.0094 - 523ms/epoch - 3ms/step
Epoch 371/500
175/175 - 1s - loss: 4.5959e-04 - val_loss: 0.0091 - 574ms/epoch - 3ms/step
Epoch 372/500
175/175 - 1s - loss: 4.3771e-04 - val_loss: 0.0092 - 513ms/epoch - 3ms/step
Epoch 373/500
175/175 - 1s - loss: 4.4189e-04 - val_loss: 0.0094 - 523ms/epoch - 3ms/step
Epoch 374/500
175/175 - 0s - loss: 4.3703e-04 - val_loss: 0.0091 - 441ms/epoch - 3ms/step
Epoch 375/500
175/175 - 1s - loss: 4.5109e-04 - val_loss: 0.0091 - 574ms/epoch - 3ms/step
Epoch 376/500
175/175 - 0s - loss: 4.5948e-04 - val_loss: 0.0092 - 431ms/epoch - 2ms/step
Epoch 377/500
175/175 - 1s - loss: 4.4516e-04 - val_loss: 0.0093 - 597ms/epoch - 3ms/step
Epoch 378/500
175/175 - 1s - loss: 4.4911e-04 - val_loss: 0.0093 - 526ms/epoch - 3ms/step
Epoch 379/500
175/175 - 

...

Epoch 125/500
175/175 - 0s - loss: 0.0000e+00 - val_loss: 0.0354 - 375ms/epoch - 2ms/step
Epoch 126/500
175/175 - 0s - loss: 0.0000e+00 - val_loss: 0.0354 - 391ms/epoch - 2ms/step
Epoch 127/500
175/175 - 0s - loss: 0.0000e+00 - val_loss: 0.0354 - 375ms/epoch - 2ms/step
Epoch 128/500
175/175 - 0s - loss: 0.0000e+00 - val_loss: 0.0354 - 281ms/epoch - 2ms/step
Epoch 129/500
175/175 - 0s - loss: 0.0000e+00 - val_loss: 0.0354 - 281ms/epoch - 2ms/step
Epoch 130/500
175/175 - 0s - loss: 0.0000e+00 - val_loss: 0.0354 - 297ms/epoch - 2ms/step
Epoch 131/500
175/175 - 0s - loss: 0.0000e+00 - val_loss: 0.0354 - 297ms/epoch - 2ms/step
Epoch 132/500
175/175 - 0s - loss: 0.0000e+00 - val_loss: 0.0354 - 312ms/epoch - 2ms/step
Epoch 133/500
175/175 - 0s - loss: 0.0000e+00 - val_loss: 0.0354 - 299ms/epoch - 2ms/step
Epoch 134/500
175/175 - 0s - loss: 0.0000e+00 - val_loss: 0.0354 - 308ms/epoch - 2ms/step
Epoch 135/500
175/175 - 0s - loss: 0.0000e+00 - val_loss: 0.0354 - 297ms/epoch - 2ms/step
Epoch 136/

175/175 - 0s - loss: 0.0000e+00 - val_loss: 0.0354 - 344ms/epoch - 2ms/step
Epoch 217/500
175/175 - 0s - loss: 0.0000e+00 - val_loss: 0.0354 - 281ms/epoch - 2ms/step
Epoch 218/500
175/175 - 0s - loss: 0.0000e+00 - val_loss: 0.0354 - 281ms/epoch - 2ms/step
Epoch 219/500
175/175 - 0s - loss: 0.0000e+00 - val_loss: 0.0354 - 297ms/epoch - 2ms/step
Epoch 220/500
175/175 - 0s - loss: 0.0000e+00 - val_loss: 0.0354 - 297ms/epoch - 2ms/step
Epoch 221/500
175/175 - 0s - loss: 0.0000e+00 - val_loss: 0.0354 - 312ms/epoch - 2ms/step
Epoch 222/500
175/175 - 0s - loss: 0.0000e+00 - val_loss: 0.0354 - 312ms/epoch - 2ms/step
Epoch 223/500
175/175 - 0s - loss: 0.0000e+00 - val_loss: 0.0354 - 297ms/epoch - 2ms/step
Epoch 224/500
175/175 - 0s - loss: 0.0000e+00 - val_loss: 0.0354 - 297ms/epoch - 2ms/step
Epoch 225/500
175/175 - 0s - loss: 0.0000e+00 - val_loss: 0.0354 - 297ms/epoch - 2ms/step
Epoch 226/500
175/175 - 0s - loss: 0.0000e+00 - val_loss: 0.0354 - 312ms/epoch - 2ms/step
Epoch 227/500
175/175 - 

Epoch 308/500
175/175 - 0s - loss: 0.0000e+00 - val_loss: 0.0354 - 297ms/epoch - 2ms/step
Epoch 309/500
175/175 - 0s - loss: 0.0000e+00 - val_loss: 0.0354 - 297ms/epoch - 2ms/step
Epoch 310/500
175/175 - 0s - loss: 0.0000e+00 - val_loss: 0.0354 - 297ms/epoch - 2ms/step
Epoch 311/500
175/175 - 0s - loss: 0.0000e+00 - val_loss: 0.0354 - 297ms/epoch - 2ms/step
Epoch 312/500
175/175 - 0s - loss: 0.0000e+00 - val_loss: 0.0354 - 306ms/epoch - 2ms/step
Epoch 313/500
175/175 - 0s - loss: 0.0000e+00 - val_loss: 0.0354 - 292ms/epoch - 2ms/step
Epoch 314/500
175/175 - 0s - loss: 0.0000e+00 - val_loss: 0.0354 - 281ms/epoch - 2ms/step
Epoch 315/500
175/175 - 0s - loss: 0.0000e+00 - val_loss: 0.0354 - 281ms/epoch - 2ms/step
Epoch 316/500
175/175 - 0s - loss: 0.0000e+00 - val_loss: 0.0354 - 312ms/epoch - 2ms/step
Epoch 317/500
175/175 - 0s - loss: 0.0000e+00 - val_loss: 0.0354 - 344ms/epoch - 2ms/step
Epoch 318/500
175/175 - 0s - loss: 0.0000e+00 - val_loss: 0.0354 - 375ms/epoch - 2ms/step
Epoch 319/

175/175 - 0s - loss: 0.0000e+00 - val_loss: 0.0354 - 312ms/epoch - 2ms/step
Epoch 400/500
175/175 - 0s - loss: 0.0000e+00 - val_loss: 0.0354 - 297ms/epoch - 2ms/step
Epoch 401/500
175/175 - 0s - loss: 0.0000e+00 - val_loss: 0.0354 - 312ms/epoch - 2ms/step
Epoch 402/500
175/175 - 0s - loss: 0.0000e+00 - val_loss: 0.0354 - 297ms/epoch - 2ms/step
Epoch 403/500
175/175 - 0s - loss: 0.0000e+00 - val_loss: 0.0354 - 297ms/epoch - 2ms/step
Epoch 404/500
175/175 - 0s - loss: 0.0000e+00 - val_loss: 0.0354 - 313ms/epoch - 2ms/step
Epoch 405/500
175/175 - 0s - loss: 0.0000e+00 - val_loss: 0.0354 - 297ms/epoch - 2ms/step
Epoch 406/500
175/175 - 0s - loss: 0.0000e+00 - val_loss: 0.0354 - 312ms/epoch - 2ms/step
Epoch 407/500
175/175 - 0s - loss: 0.0000e+00 - val_loss: 0.0354 - 297ms/epoch - 2ms/step
Epoch 408/500
175/175 - 0s - loss: 0.0000e+00 - val_loss: 0.0354 - 312ms/epoch - 2ms/step
Epoch 409/500
175/175 - 0s - loss: 0.0000e+00 - val_loss: 0.0354 - 297ms/epoch - 2ms/step
Epoch 410/500
175/175 - 

Epoch 491/500
175/175 - 0s - loss: 0.0000e+00 - val_loss: 0.0354 - 281ms/epoch - 2ms/step
Epoch 492/500
175/175 - 0s - loss: 0.0000e+00 - val_loss: 0.0354 - 297ms/epoch - 2ms/step
Epoch 493/500
175/175 - 0s - loss: 0.0000e+00 - val_loss: 0.0354 - 297ms/epoch - 2ms/step
Epoch 494/500
175/175 - 0s - loss: 0.0000e+00 - val_loss: 0.0354 - 297ms/epoch - 2ms/step
Epoch 495/500
175/175 - 0s - loss: 0.0000e+00 - val_loss: 0.0354 - 297ms/epoch - 2ms/step
Epoch 496/500
175/175 - 0s - loss: 0.0000e+00 - val_loss: 0.0354 - 297ms/epoch - 2ms/step
Epoch 497/500
175/175 - 0s - loss: 0.0000e+00 - val_loss: 0.0354 - 297ms/epoch - 2ms/step
Epoch 498/500
175/175 - 0s - loss: 0.0000e+00 - val_loss: 0.0354 - 312ms/epoch - 2ms/step
Epoch 499/500
175/175 - 0s - loss: 0.0000e+00 - val_loss: 0.0354 - 297ms/epoch - 2ms/step
Epoch 500/500
175/175 - 0s - loss: 0.0000e+00 - val_loss: 0.0354 - 297ms/epoch - 2ms/step
Train: 0.000, Test: 0.035
Modelo de SAN PEDRO DEL YCUAMANDIYU guardado
trainScore 3.7681672662530624

Dataframe con los resultados

In [13]:
data_tuples = list(zip(Dist,MSEtrain,MSETest,Registros))
Tabla = pd.DataFrame(data_tuples, columns =['Departamento', 'Train Score', 'Test Score', 'Registros']) 
# print the data 
Tabla

,Departamento,Train Score,Test Score,Registros
0,ASUNCION,7.445078e-03,0.036217,265
1,FERNANDO DE LA MORA,9.063332e-03,0.033673,265
2,SAN LORENZO,4.465467e-03,0.089616,265
3,FUERTE OLIMPO,3.600770e-03,0.005112,265
4,PEDRO JUAN CABALLERO,3.100516e-02,0.021098,265
5,FILADELFIA,2.114727e-03,0.005098,265
6,CORONEL OVIEDO,3.652122e-03,0.009865,265
7,CAAZAPA,2.046196e-03,0.003141,265
8,SALTO DEL GUAIRA,4.122349e-03,0.017135,265
9,AREGUA,4.603889e-03,0.126878,265


Guardamos estos resultados en un .csv

In [14]:
Tabla.to_csv("./data/Resultados/Distritos/Resultados_"+str(dist)+".csv")

## Por clústers

Clusters

In [15]:
#Cluster 1
C1=['1RO DE MARZO'
,'25 DE DICIEMBRE'
,'ANTEQUERA'
,'AYOLAS'
,'CABALLERO ALVAREZ'
,'CAPIIBARY'
,'CAPITAN MEZA'
,'CARAYAO'
,'CARLOS ANTONIO LOPEZ'
,'CECILIO BAEZ'
,'EUGENIO A GARAY'
,'ISLA PUCU'
,'ITACURUBI DEL ROSARIO'
,'ITURBE'
,'LIMOY PUEBLO'
,'LORETO'
,'MACIEL'
,'NUEVA GERMANIA'
,'SAN CARLOS'
,'SAN COSME Y DAMIAN'
,'SANTA ROSA DEL AGUARAY'
,'SANTA ROSA DEL MBUTUY'
,'VALENZUELA'
,'VILLA DEL ROSARIO'
,'YATAITY DEL NORTE'
,'YGUAZU'
,'YHU']
#Cluster 2
C2=['3 DE FEBRERO'
,'ABAI'
,'BAHIA NEGRA'
,'CAPITAN MIRANDA'
,'COLONIA INDEPENDENCIA'
,'DESMOCHADOS'
,'DR BOTRELL'
,'GENERAL ARTIGAS'
,'GENERAL DELGADO'
,'GENERAL DIAZ'
,'HERNANDARIAS'
,'HUMAITA'
,'ITAPUA POTY'
,'JESUS'
,'LA PASTORA'
,'LAURELES'
,'MCAL. FRANCISCO SOLANO LOPEZ'
,'MINGA GUAZU'
,'NUEVA ALBORADA'
,'NUEVA COLOMBIA'
,'NUEVA ESPERANZA'
,'NUEVA LONDRES'
,'PEDRO JUAN CABALLERO'
,'SAN JOSE OBRERO'
,'SAN LAZARO'
,'SAN MIGUEL'
,'SANTA MARIA'
,'SIMON BOLIVAR'
,'TACUARAS'
,'TACUATI'
,'TEMBIAPORA'
,'TRINIDAD'
,'VILLA OLIVA'
,'YEGROS']
#Cluster 3
C3=['ACAHAY'
,'ALTOS'
,'AREGUA'
,'ARROYOS Y ESTEROS'
,'ATYRA'
,'BELLA VISTA'
,'BENJAMIN ACEVAL'
,'CAACUPE'
,'CAAGUAZU'
,'CAAZAPA'
,'CAMBYRETA'
,'CAPIATA'
,'CAPITAN BADO'
,'CARAGUATAY'
,'CHORE'
,'EMBOSCADA'
,'ENCARNACION'
,'EUSEBIO AYALA'
,'FASSARDI'
,'FERNANDO DE LA MORA'
,'GUARAMBARE'
,'ITA'
,'ITACURUBI DE LA CORDILLERA'
,'ITAPE'
,'ITAUGUA'
,'J A SALDIVAR'
,'JUAN E. OLEARY'
,'JUAN LEON MALLORQUIN'
,'KATUETE'
,'LA PALOMA'
,'LAMBARE'
,'LOMA GRANDE'
,'LOMA PLATA'
,'LUQUE'
,'MARIANO ROQUE ALONSO'
,'NATALICIO TALAVERA'
,'PIRAYU'
,'PIRIBEBUY'
,'QUIINDY'
,'REPATRIACION'
,'SALTO DEL GUAIRA'
,'SAN ANTONIO'
,'SAN BERNARDINO'
,'SAN ESTANISLAO'
,'SAN IGNACIO'
,'SAN LORENZO'
,'SAN PEDRO DEL PARANA'
,'SANTA ELENA'
,'TAVAPY'
,'TOBATI'
,'VILLA ELISA'
,'VILLA HAYES'
,'VILLETA'
,'YPACARAI'
,'YPANE'
,'ZANJA PYTA']
#Cluster 4
C4=['ALBERDI'
,'ASUNCION'
,'BORJA'
,'BUENA VISTA'
,'CARAPEGUA'
,'CARMEN DEL PARANA'
,'CORONEL OVIEDO'
,'CORPUS CHRISTI'
,'CURUGUATY'
,'GENERAL AQUINO'
,'GUAZUCUA'
,'ISLA UMBU'
,'JUAN DE MENA'
,'LIMPIO'
,'NANAWA'
,'PARAGUARI'
,'PILAR'
,'PIRAPO'
,'SAN JOAQUIN'
,'SAN JOSE DE LOS ARROYOS'
,'SAN PATRICIO'
,'SAN PEDRO DEL YCUAMANDIYU'
,'TAVAI'
,'TEBICUARY'
,'VILLARRICA'
,'YAGUARON'
,'YATYTAY'
,'YPEJHU'
,'YUTY']
#Cluster 5
C5=['ALTO VERA'
,'AZOTEY'
,'CERRITO'
,'CORONEL BOGADO'
,'DR. JUAN MANUEL FRUTOS'
,'ESCOBAR'
,'FILADELFIA'
,'GENERAL BERNARDINO CABALLERO'
,'GENERAL BRUGUEZ'
,'GENERAL MORINIGO'
,'HOHENAU'
,'ITAKYRY'
,'LEANDRO OVIEDO'
,'MBOCAYATY DEL YHAGUY'
,'MCAL. ESTIGARRIBIA'
,'NATALIO'
,'NUEVA ITALIA'
,'OBLIGADO'
,'PUERTO FALCON'
,'RAUL ARSENIO OVIEDO'
,'ROQUE GONZALEZ DE SANTA CRUZ'
,'SAN JUAN DEL PARANA'
,'SAN RAFAEL DEL PARANA'
,'SAN SALVADOR'
,'SANTA RITA'
,'SANTIAGO'
,'SAPUCAI'
,'TOMAS ROMERO PEREIRA'
,'VAQUERIA'
,'VILLALBIN'
,'YATAITY'
,'YBY YAU'
,'YBYTYMI']
#Cluster 6
C6=['BELEN'
,'FUERTE OLIMPO'
,'LIMA']
#Cluster 7
C7=['CARMELO PERALTA'
,'CHACO'
,'LA VICTORIA'
,'POZO COLORADO'
,'SAN ROQUE GONZALEZ DE SANTACRUZ'
,'YBYRAROVANA']
#Cluster 8
C8=['COLONIA FRAM'
,'CONCEPCION'
,'CORONEL MARTINEZ'
,'EDELIRA'
,'FELIX PEREZ CARDOZO'
,'GUAYAIBI'
,'HORQUETA'
,'ITANARA'
,'JUAN EULOGIO ESTIGARRIBIA'
,'MAURICIO JOSE TROCHE'
,'MBOCAYATY'
,'MOISES BERTONI'
,'PASO YOBAI'
,'SAN JUAN BAUTISTA'
,'SAN JUAN NEPOMUCENO'
,'SAN PEDRO'
,'SANTA ROSA'
,'SANTA ROSA DEL MONDAY'
,'TEBICUARYMI'
,'UNION'
,'YGATIMI']
#Cluster 9
C9=['GENERAL RESQUIN'
,'JOSE DOMINGO OCAMPOS'
,'MBARACAYU'
,'MINGA PORA'
,'NARANJAL'
,'PUERTO PINASCO'
,'R I 3 CORRALES'
,'SAN ALBERTO']

In [ ]:
lista_clusters=[C3,C4,C5,C6,C8,C2]
MSEtrain=[]
MSETest=[]
Registros=[]
Dist=[]
dist=1
for C in lista_clusters:
    #print(C)
    datasetxdist=dataset[dataset['distrito_nombre'].isin(C)]
    #print(dist)
    #distrito=dataset['distrito_nombre']==dist
    #datasetxdist=dataset[distrito]
    print("Cluster: ",C)
    trainScore,testScore,reg=distLSTM(datasetxdist)
    print("trainScore",trainScore)
    print("testScore",testScore)
    print("reg",reg)
    #Para el dataframe
    MSEtrain.append(trainScore)
    MSETest.append(testScore)
    Registros.append(reg)
    Dist.append(dist)
    data_tuples = list(zip(Dist,MSEtrain,MSETest,Registros))
    Tabla = pd.DataFrame(data_tuples, columns =['Distrito', 'Train Score', 'Test Score', 'Registros']) 
    dist=dist+1

Cluster:  ['ACAHAY', 'ALTOS', 'AREGUA', 'ARROYOS Y ESTEROS', 'ATYRA', 'BELLA VISTA', 'BENJAMIN ACEVAL', 'CAACUPE', 'CAAGUAZU', 'CAAZAPA', 'CAMBYRETA', 'CAPIATA', 'CAPITAN BADO', 'CARAGUATAY', 'CHORE', 'EMBOSCADA', 'ENCARNACION', 'EUSEBIO AYALA', 'FASSARDI', 'FERNANDO DE LA MORA', 'GUARAMBARE', 'ITA', 'ITACURUBI DE LA CORDILLERA', 'ITAPE', 'ITAUGUA', 'J A SALDIVAR', 'JUAN E. OLEARY', 'JUAN LEON MALLORQUIN', 'KATUETE', 'LA PALOMA', 'LAMBARE', 'LOMA GRANDE', 'LOMA PLATA', 'LUQUE', 'MARIANO ROQUE ALONSO', 'NATALICIO TALAVERA', 'PIRAYU', 'PIRIBEBUY', 'QUIINDY', 'REPATRIACION', 'SALTO DEL GUAIRA', 'SAN ANTONIO', 'SAN BERNARDINO', 'SAN ESTANISLAO', 'SAN IGNACIO', 'SAN LORENZO', 'SAN PEDRO DEL PARANA', 'SANTA ELENA', 'TAVAPY', 'TOBATI', 'VILLA ELISA', 'VILLA HAYES', 'VILLETA', 'YPACARAI', 'YPANE', 'ZANJA PYTA']
9942 4898
Epoch 1/500
9940/9940 - 18s - loss: 6.1703e-04 - val_loss: 1.8257e-04 - 18s/epoch - 2ms/step
Epoch 2/500
9940/9940 - 16s - loss: 2.1657e-04 - val_loss: 1.7199e-04 - 16s/epoch 

Epoch 80/500
9940/9940 - 16s - loss: 2.1860e-04 - val_loss: 1.8313e-04 - 16s/epoch - 2ms/step
Epoch 81/500
9940/9940 - 16s - loss: 2.1735e-04 - val_loss: 1.6781e-04 - 16s/epoch - 2ms/step
Epoch 82/500
9940/9940 - 16s - loss: 2.1065e-04 - val_loss: 1.8049e-04 - 16s/epoch - 2ms/step
Epoch 83/500
9940/9940 - 16s - loss: 2.1619e-04 - val_loss: 1.6819e-04 - 16s/epoch - 2ms/step
Epoch 84/500
9940/9940 - 15s - loss: 2.1493e-04 - val_loss: 1.8026e-04 - 15s/epoch - 2ms/step
Epoch 85/500
9940/9940 - 16s - loss: 2.1501e-04 - val_loss: 1.7388e-04 - 16s/epoch - 2ms/step
Epoch 86/500
9940/9940 - 16s - loss: 2.1593e-04 - val_loss: 1.7638e-04 - 16s/epoch - 2ms/step
Epoch 87/500
9940/9940 - 16s - loss: 2.1637e-04 - val_loss: 1.9263e-04 - 16s/epoch - 2ms/step
Epoch 88/500
9940/9940 - 16s - loss: 2.1296e-04 - val_loss: 1.7987e-04 - 16s/epoch - 2ms/step
Epoch 89/500
9940/9940 - 16s - loss: 2.1343e-04 - val_loss: 1.6744e-04 - 16s/epoch - 2ms/step
Epoch 90/500
9940/9940 - 16s - loss: 2.1294e-04 - val_loss: 

Epoch 167/500
9940/9940 - 24s - loss: 2.1429e-04 - val_loss: 1.8945e-04 - 24s/epoch - 2ms/step
Epoch 168/500
9940/9940 - 21s - loss: 2.1347e-04 - val_loss: 2.1482e-04 - 21s/epoch - 2ms/step
Epoch 169/500
9940/9940 - 19s - loss: 2.1067e-04 - val_loss: 1.6807e-04 - 19s/epoch - 2ms/step
Epoch 170/500
9940/9940 - 21s - loss: 2.1565e-04 - val_loss: 1.6727e-04 - 21s/epoch - 2ms/step
Epoch 171/500
9940/9940 - 18s - loss: 2.1370e-04 - val_loss: 1.9027e-04 - 18s/epoch - 2ms/step
Epoch 172/500
9940/9940 - 20s - loss: 2.1396e-04 - val_loss: 1.8756e-04 - 20s/epoch - 2ms/step
Epoch 173/500
9940/9940 - 26s - loss: 2.1500e-04 - val_loss: 1.9661e-04 - 26s/epoch - 3ms/step
Epoch 174/500
9940/9940 - 20s - loss: 2.1469e-04 - val_loss: 3.0373e-04 - 20s/epoch - 2ms/step
Epoch 175/500
9940/9940 - 26s - loss: 2.1237e-04 - val_loss: 1.7110e-04 - 26s/epoch - 3ms/step
Epoch 176/500
9940/9940 - 20s - loss: 2.1063e-04 - val_loss: 1.6850e-04 - 20s/epoch - 2ms/step
Epoch 177/500
9940/9940 - 19s - loss: 2.1314e-04 -

Epoch 254/500
9940/9940 - 21s - loss: 2.1182e-04 - val_loss: 1.7390e-04 - 21s/epoch - 2ms/step
Epoch 255/500
9940/9940 - 21s - loss: 2.1208e-04 - val_loss: 1.8366e-04 - 21s/epoch - 2ms/step
Epoch 256/500
9940/9940 - 23s - loss: 2.1336e-04 - val_loss: 1.6963e-04 - 23s/epoch - 2ms/step
Epoch 257/500
9940/9940 - 20s - loss: 2.1077e-04 - val_loss: 1.6931e-04 - 20s/epoch - 2ms/step
Epoch 258/500
9940/9940 - 21s - loss: 2.1421e-04 - val_loss: 1.9419e-04 - 21s/epoch - 2ms/step
Epoch 259/500
9940/9940 - 24s - loss: 2.1314e-04 - val_loss: 1.6986e-04 - 24s/epoch - 2ms/step
Epoch 260/500
9940/9940 - 20s - loss: 2.1147e-04 - val_loss: 1.6797e-04 - 20s/epoch - 2ms/step
Epoch 261/500
9940/9940 - 22s - loss: 2.1057e-04 - val_loss: 2.5652e-04 - 22s/epoch - 2ms/step
Epoch 262/500
9940/9940 - 24s - loss: 2.1136e-04 - val_loss: 1.7226e-04 - 24s/epoch - 2ms/step
Epoch 263/500
9940/9940 - 23s - loss: 2.0855e-04 - val_loss: 1.7142e-04 - 23s/epoch - 2ms/step
Epoch 264/500
9940/9940 - 24s - loss: 2.1437e-04 -

Epoch 341/500
9940/9940 - 17s - loss: 2.1313e-04 - val_loss: 1.6781e-04 - 17s/epoch - 2ms/step
Epoch 342/500
9940/9940 - 17s - loss: 2.1460e-04 - val_loss: 1.6921e-04 - 17s/epoch - 2ms/step
Epoch 343/500
9940/9940 - 17s - loss: 2.1126e-04 - val_loss: 1.6837e-04 - 17s/epoch - 2ms/step
Epoch 344/500
9940/9940 - 17s - loss: 2.1437e-04 - val_loss: 1.8169e-04 - 17s/epoch - 2ms/step
Epoch 345/500
9940/9940 - 18s - loss: 2.1328e-04 - val_loss: 1.6771e-04 - 18s/epoch - 2ms/step
Epoch 346/500
9940/9940 - 17s - loss: 2.1061e-04 - val_loss: 1.7594e-04 - 17s/epoch - 2ms/step
Epoch 347/500
9940/9940 - 17s - loss: 2.0990e-04 - val_loss: 1.6803e-04 - 17s/epoch - 2ms/step
Epoch 348/500
9940/9940 - 17s - loss: 2.1363e-04 - val_loss: 1.7009e-04 - 17s/epoch - 2ms/step
Epoch 349/500
9940/9940 - 17s - loss: 2.0141e-04 - val_loss: 1.7259e-04 - 17s/epoch - 2ms/step
Epoch 350/500
9940/9940 - 17s - loss: 2.1484e-04 - val_loss: 1.7308e-04 - 17s/epoch - 2ms/step
Epoch 351/500
9940/9940 - 18s - loss: 2.1345e-04 -

...

3726/3726 - 6s - loss: 0.0015 - val_loss: 6.1971e-04 - 6s/epoch - 2ms/step
Epoch 220/500
3726/3726 - 6s - loss: 0.0015 - val_loss: 5.9792e-04 - 6s/epoch - 2ms/step
Epoch 221/500
3726/3726 - 6s - loss: 0.0015 - val_loss: 7.3038e-04 - 6s/epoch - 2ms/step
Epoch 222/500
3726/3726 - 6s - loss: 0.0015 - val_loss: 7.1041e-04 - 6s/epoch - 2ms/step
Epoch 223/500
3726/3726 - 6s - loss: 0.0015 - val_loss: 6.1191e-04 - 6s/epoch - 2ms/step
Epoch 224/500
3726/3726 - 6s - loss: 0.0015 - val_loss: 6.2586e-04 - 6s/epoch - 2ms/step
Epoch 225/500
3726/3726 - 6s - loss: 0.0015 - val_loss: 5.9795e-04 - 6s/epoch - 2ms/step
Epoch 226/500
3726/3726 - 6s - loss: 0.0015 - val_loss: 6.2292e-04 - 6s/epoch - 2ms/step
Epoch 227/500
3726/3726 - 6s - loss: 0.0015 - val_loss: 6.5244e-04 - 6s/epoch - 2ms/step
Epoch 228/500
3726/3726 - 6s - loss: 0.0015 - val_loss: 8.6212e-04 - 6s/epoch - 2ms/step
Epoch 229/500
3726/3726 - 6s - loss: 0.0015 - val_loss: 6.7911e-04 - 6s/epoch - 2ms/step
Epoch 230/500
3726/3726 - 6s - loss

Epoch 312/500
3726/3726 - 6s - loss: 0.0015 - val_loss: 6.4220e-04 - 6s/epoch - 2ms/step
Epoch 313/500
3726/3726 - 6s - loss: 0.0015 - val_loss: 6.0924e-04 - 6s/epoch - 2ms/step
Epoch 314/500
3726/3726 - 6s - loss: 0.0015 - val_loss: 6.0733e-04 - 6s/epoch - 2ms/step
Epoch 315/500
3726/3726 - 6s - loss: 0.0015 - val_loss: 6.1689e-04 - 6s/epoch - 2ms/step
Epoch 316/500
3726/3726 - 6s - loss: 0.0015 - val_loss: 6.2562e-04 - 6s/epoch - 2ms/step
Epoch 317/500
3726/3726 - 7s - loss: 0.0015 - val_loss: 6.0644e-04 - 7s/epoch - 2ms/step
Epoch 318/500
3726/3726 - 6s - loss: 0.0015 - val_loss: 6.8435e-04 - 6s/epoch - 2ms/step
Epoch 319/500
3726/3726 - 6s - loss: 0.0015 - val_loss: 6.2068e-04 - 6s/epoch - 2ms/step
Epoch 320/500
3726/3726 - 6s - loss: 0.0015 - val_loss: 6.6625e-04 - 6s/epoch - 2ms/step
Epoch 321/500
3726/3726 - 6s - loss: 0.0015 - val_loss: 6.0499e-04 - 6s/epoch - 2ms/step
Epoch 322/500
3726/3726 - 6s - loss: 0.0015 - val_loss: 6.3533e-04 - 6s/epoch - 2ms/step
Epoch 323/500
3726/37

3726/3726 - 8s - loss: 0.0015 - val_loss: 6.1280e-04 - 8s/epoch - 2ms/step
Epoch 405/500
3726/3726 - 8s - loss: 0.0015 - val_loss: 6.1102e-04 - 8s/epoch - 2ms/step
Epoch 406/500
3726/3726 - 8s - loss: 0.0015 - val_loss: 6.0755e-04 - 8s/epoch - 2ms/step
Epoch 407/500
3726/3726 - 8s - loss: 0.0015 - val_loss: 6.4157e-04 - 8s/epoch - 2ms/step
Epoch 408/500
3726/3726 - 7s - loss: 0.0015 - val_loss: 6.6069e-04 - 7s/epoch - 2ms/step
Epoch 409/500
3726/3726 - 7s - loss: 0.0015 - val_loss: 6.3398e-04 - 7s/epoch - 2ms/step
Epoch 410/500
3726/3726 - 8s - loss: 0.0015 - val_loss: 6.1170e-04 - 8s/epoch - 2ms/step
Epoch 411/500
3726/3726 - 8s - loss: 0.0015 - val_loss: 6.6332e-04 - 8s/epoch - 2ms/step
Epoch 412/500
3726/3726 - 7s - loss: 0.0015 - val_loss: 6.2694e-04 - 7s/epoch - 2ms/step
Epoch 413/500
3726/3726 - 8s - loss: 0.0015 - val_loss: 6.0926e-04 - 8s/epoch - 2ms/step
Epoch 414/500
3726/3726 - 7s - loss: 0.0015 - val_loss: 6.0278e-04 - 7s/epoch - 2ms/step
Epoch 415/500
3726/3726 - 7s - loss

Epoch 497/500
3726/3726 - 7s - loss: 0.0014 - val_loss: 6.2314e-04 - 7s/epoch - 2ms/step
Epoch 498/500
3726/3726 - 7s - loss: 0.0015 - val_loss: 6.2281e-04 - 7s/epoch - 2ms/step
Epoch 499/500
3726/3726 - 7s - loss: 0.0015 - val_loss: 6.1154e-04 - 7s/epoch - 2ms/step
Epoch 500/500
3726/3726 - 7s - loss: 0.0015 - val_loss: 6.0112e-04 - 7s/epoch - 2ms/step
Train: 0.001, Test: 0.001
Modelo de 5 guardado
trainScore 0.006326920568785447
testScore 0.004135867045035543
reg 5565
Cluster:  ['3 DE FEBRERO', 'ABAI', 'BAHIA NEGRA', 'CAPITAN MIRANDA', 'COLONIA INDEPENDENCIA', 'DESMOCHADOS', 'DR BOTRELL', 'GENERAL ARTIGAS', 'GENERAL DELGADO', 'GENERAL DIAZ', 'HERNANDARIAS', 'HUMAITA', 'ITAPUA POTY', 'JESUS', 'LA PASTORA', 'LAURELES', 'MCAL. FRANCISCO SOLANO LOPEZ', 'MINGA GUAZU', 'NUEVA ALBORADA', 'NUEVA COLOMBIA', 'NUEVA ESPERANZA', 'NUEVA LONDRES', 'PEDRO JUAN CABALLERO', 'SAN JOSE OBRERO', 'SAN LAZARO', 'SAN MIGUEL', 'SANTA MARIA', 'SIMON BOLIVAR', 'TACUARAS', 'TACUATI', 'TEMBIAPORA', 'TRINIDAD', 

Epoch 78/500
6034/6034 - 11s - loss: 1.9516e-04 - val_loss: 4.7732e-05 - 11s/epoch - 2ms/step
Epoch 79/500
6034/6034 - 12s - loss: 1.9036e-04 - val_loss: 4.5931e-05 - 12s/epoch - 2ms/step
Epoch 80/500
6034/6034 - 12s - loss: 1.9599e-04 - val_loss: 5.1213e-05 - 12s/epoch - 2ms/step
Epoch 81/500
6034/6034 - 12s - loss: 1.9849e-04 - val_loss: 5.1776e-05 - 12s/epoch - 2ms/step
Epoch 82/500
6034/6034 - 11s - loss: 1.9754e-04 - val_loss: 5.1305e-05 - 11s/epoch - 2ms/step
Epoch 83/500
6034/6034 - 11s - loss: 1.9218e-04 - val_loss: 4.8632e-05 - 11s/epoch - 2ms/step
Epoch 84/500
6034/6034 - 11s - loss: 1.9288e-04 - val_loss: 4.6746e-05 - 11s/epoch - 2ms/step
Epoch 85/500
6034/6034 - 12s - loss: 1.9914e-04 - val_loss: 4.7354e-05 - 12s/epoch - 2ms/step
Epoch 86/500
6034/6034 - 11s - loss: 1.9650e-04 - val_loss: 4.8543e-05 - 11s/epoch - 2ms/step
Epoch 87/500
6034/6034 - 12s - loss: 1.9518e-04 - val_loss: 5.0250e-05 - 12s/epoch - 2ms/step
Epoch 88/500
6034/6034 - 11s - loss: 1.9409e-04 - val_loss: 

Dataframe con los resultados

In [ ]:
data_tuples = list(zip(Dist,MSEtrain,MSETest,Registros))
Tabla = pd.DataFrame(data_tuples, columns =['Departamento', 'Train Score', 'Test Score', 'Registros']) 
# print the data 
Tabla

In [ ]:
Guardamos en un csv

In [ ]:
Tabla.to_csv("./data/Resultados/Clusters/Resultados_clusters.csv")